# Лабораторная работа 2: Линейная и логистическая регрессия

## Шаг 1: Выбор начальных условий

### 1a. Датасет для классификации
- **Набор данных:** Iris
- **Обоснование:** Небольшой, сбалансированный датасет для классификации.

### 1b. Датасет для регрессии
- **Набор данных:** California Housing
- **Обоснование:** Реальные данные о ценах на жильё, подходят для регрессии.

### 1c. Метрики качества
- **Классификация:** Accuracy
- **Обоснование:** Accuracy выбрана как простая и эффективная метрика для оценки качества моделей на сбалансированном датасете Iris.
- **Регрессия:** Mean Squared Error (MSE), R²
- **Обоснование:** MSE и R² выбраны для оценки моделей регрессии, так как они дают количественную и качественную оценку предсказаний модели на датасете California Housing.
-Эти метрики в совокупности обеспечивают всестороннюю оценку моделей KNN как для классификации, так и для регрессии.

In [10]:
# Подготовка данных
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
import pandas as pd
import numpy as np

###  Шаг 2: Создание бейзлайна и оценка качества

In [11]:
# Генерация данных (замените этот блок на загрузку данных из лабораторной работы 1)
np.random.seed(42)
X_reg = np.random.rand(100, 3) * 100
y_reg = 2 * X_reg[:, 0] + 3 * X_reg[:, 1] - X_reg[:, 2] + np.random.randn(100) * 10

X_class = np.random.rand(100, 3) * 100
y_class = (X_class[:, 0] + X_class[:, 1] > X_class[:, 2]).astype(int)

# Разделение на тренировочную и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

# Масштабирование данных
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

scaler_class = StandardScaler()
X_train_class_scaled = scaler_class.fit_transform(X_train_class)
X_test_class_scaled = scaler_class.transform(X_test_class)

In [12]:
# Линейная регрессия
lin_reg = LinearRegression()
lin_reg.fit(X_train_reg_scaled, y_train_reg)
y_pred_reg = lin_reg.predict(X_test_reg_scaled)
mse_reg = mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

In [13]:
# Логистическая регрессия
log_reg = LogisticRegression()
log_reg.fit(X_train_class_scaled, y_train_class)
y_pred_class = log_reg.predict(X_test_class_scaled)
accuracy_class = accuracy_score(y_test_class, y_pred_class)

In [14]:
# Вывод результатов
results = {
    "Линейная регрессия": {
        "MSE": mse_reg,
        "R²": r2_reg
    },
    "Логистическая регрессия": {
        "Accuracy": accuracy_class
    }
}

results_df = pd.DataFrame(results)
results_df

,Линейная регрессия,Логистическая регрессия
MSE,201.739248,NaN
R²,0.984106,NaN
Accuracy,NaN,0.95


##  3. Улучшение бейзлайна

###  3a. Сформулировать гипотезы
- **Препроцессинг данных:** Нормализация, удаление выбросов.
- **Визуализация данных:** Исследование распределений и зависимостей.
- **Новые признаки:** Создание дополнительных признаков.
- **Оптимизация гиперпараметров:** Подбор лучших значений через GridSearchCV.

###3b. Проверить гипотезы
Реализуем препроцессинг, визуализацию и оптимизацию гиперпараметров.

In [15]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
# Полиномиальные признаки для регрессии
poly = PolynomialFeatures(degree=2)
X_train_reg_poly = poly.fit_transform(X_train_reg_scaled)
X_test_reg_poly = poly.transform(X_test_reg_scaled)
# Линейная регрессия
lin_reg = LinearRegression()
lin_reg.fit(X_train_reg_poly, y_train_reg)
y_pred_reg = lin_reg.predict(X_test_reg_poly)
mse_reg = mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

# Логистическая регрессия
log_reg = LogisticRegression(max_iter=1000)
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid_log_reg = GridSearchCV(log_reg, param_grid, cv=5)
grid_log_reg.fit(X_train_class_scaled, y_train_class)
best_log_reg = grid_log_reg.best_estimator_
y_pred_class = best_log_reg.predict(X_test_class_scaled)
accuracy_class = accuracy_score(y_test_class, y_pred_class)

###3c. Сформировать улучшенный бейзлайн

In [16]:
print("Регрессия:")
print(f"MSE: {mse_reg:.4f}, R^2: {r2_reg:.4f}")
print("Классификация:")
print(f"Accuracy: {accuracy_class:.4f}")

Регрессия:
MSE: 210.6077, R^2: 0.9834
Классификация:
Accuracy: 0.9500


###  3d. Обучить модели с улучшенным бейзлайном

In [17]:
# Обучение моделей с улучшенными параметрами
lin_reg.fit(X_train_reg_poly, y_train_reg)
best_log_reg.fit(X_train_class_scaled, y_train_class)

LogisticRegression(C=100, max_iter=1000)

###3e. Оценить качество моделей

In [18]:
mse_reg_improved = mean_squared_error(y_test_reg, lin_reg.predict(X_test_reg_poly))
r2_reg_improved = r2_score(y_test_reg, lin_reg.predict(X_test_reg_poly))
accuracy_class_improved = accuracy_score(y_test_class, best_log_reg.predict(X_test_class_scaled))

print("Improved Regression Metrics:")
print(f"MSE: {mse_reg_improved:.4f}, R^2: {r2_reg_improved:.4f}")
print("Improved Classification Metrics:")
print(f"Accuracy: {accuracy_class_improved:.4f}")

Improved Regression Metrics:
MSE: 210.6077, R^2: 0.9834
Improved Classification Metrics:
Accuracy: 0.9500


###  3f. Сравнить результаты с бейзлайном

| **Задача**       | **Метрика** | **Базовый бейзлайн** | **Улучшенный бейзлайн** | **Изменение** |
|------------------|------------|---------------------|------------------------|-------------|
| **Классификация** | Accuracy   | 0.95               | 0.95                  | ➖          |
| **Регрессия**    | MSE        | 201.739248               | 210.6077                  | 📉 Ухудшение |
| **Регрессия**    | R²         | 0.984106               | 0.983468                  | 📈 Ухудшение |

###  3g. Выводы
- Базовый и улучшенный бейзлайны показали одинаковый результат точности — 0.95.
 Улучшения  не повлияли на качество классификации.
- Базовый MSE: 201.7392, улучшенный MSE: 210.6077.Улучшения привели к ухудшению метрики. Возможно, модель переобучилась или добавленные признаки внесли больше шума.
- Базовый R²: 0.9841, улучшенный R²: 0.9834.Незначительное снижение качества. Это может быть связано с тем, что оптимизация гиперпараметров не нашла лучшую комбинацию параметров или данные недостаточно обработаны.

##4. Имплементация алгоритмов машинного обучения

###4a. Самостоятельно имплементировать алгоритмы машинного обучения
Реализуем алгоритм **K-Nearest Neighbors (KNN)** вручную для классификации и регрессии.

In [19]:
# Реализация линейной регрессии
class MyLinearRegression:
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X, y):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        theta_best = np.linalg.pinv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
        self.intercept_ = theta_best[0]
        self.coef_ = theta_best[1:]

    def predict(self, X):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return X_b.dot(np.r_[self.intercept_, self.coef_])

# Реализация логистической регрессии
class MyLogisticRegression:
    def __init__(self, lr=0.01, n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter
        self.theta = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        self.theta = np.zeros(X_b.shape[1])
        for _ in range(self.n_iter):
            gradients = X_b.T.dot(self.sigmoid(X_b.dot(self.theta)) - y) / len(y)
            self.theta -= self.lr * gradients

    def predict(self, X):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return (self.sigmoid(X_b.dot(self.theta)) >= 0.5).astype(int)

###4b. Обучить имплементированные модели

In [20]:
# Обучение
my_lin_reg = MyLinearRegression()
my_lin_reg.fit(X_train_reg_scaled, y_train_reg)
y_pred_reg_custom = my_lin_reg.predict(X_test_reg_scaled)

my_log_reg = MyLogisticRegression(lr=0.1, n_iter=1000)
my_log_reg.fit(X_train_class_scaled, y_train_class)
y_pred_class_custom = my_log_reg.predict(X_test_class_scaled)

###4c. Оценить качество имплементированных моделей

In [21]:
mse_custom = mean_squared_error(y_test_reg, y_pred_reg_custom)
r2_custom = r2_score(y_test_reg, y_pred_reg_custom)
accuracy_custom = accuracy_score(y_test_class, y_pred_class_custom)

print("Custom Regression:")
print(f"MSE: {mse_custom:.4f}, R^2: {r2_custom:.4f}")
print("Custom Classification:")
print(f"Accuracy: {accuracy_custom:.4f}")

Custom Regression:
MSE: 201.7392, R^2: 0.9841
Custom Classification:
Accuracy: 0.9500


###4d. Сравнить результаты имплементированных моделей с пунктом 2

| **Задача**       | **Метрика** | **Базовый бейзлайн** | **Custom KNN** | **Изменение** |
|------------------|------------|---------------------|---------------|-------------|
| **Классификация** | Accuracy   | 0.95               | 0.95          | ➖          |
| **Регрессия**    | MSE        | 201.739248               | 201.7392          | ➖  |
| **Регрессия**    | R²         | 0.984106               | 0.9841          | ➖  |


###4e. Выводы
Кастомная реализация алгоритмов машинного обучения продемонстрировала аналогичные результаты, что подтверждает корректность её выполнения

###4f. Применение техник из улучшенного бейзлайна (пункт 3c)

Применяем техники из улучшенного бейзлайна, включая нормализацию данных и оптимизацию гиперпараметров.

In [22]:
# Использование полиномиальных признаков и оптимизированных гиперпараметров
X_train_reg_poly = poly.fit_transform(X_train_reg_scaled)
X_test_reg_poly = poly.transform(X_test_reg_scaled)
best_log_reg.fit(X_train_class_scaled, y_train_class)


LogisticRegression(C=100, max_iter=1000)

###4g. Обучить модели (для классификации и регрессии) для улучшенных данных

In [23]:
# Обучение моделей с улучшенными техниками
lin_reg.fit(X_train_reg_poly, y_train_reg)
best_log_reg.fit(X_train_class_scaled, y_train_class)

LogisticRegression(C=100, max_iter=1000)

###4h. Оценить качество моделей (для классификации и регрессии)

In [24]:
# Повторная оценка качества моделей после применения улучшенных техник
mse_reg_final = mean_squared_error(y_test_reg, lin_reg.predict(X_test_reg_poly))
r2_reg_final = r2_score(y_test_reg, lin_reg.predict(X_test_reg_poly))
accuracy_class_final = accuracy_score(y_test_class, best_log_reg.predict(X_test_class_scaled))

print("Final Regression Metrics:")
print(f"MSE: {mse_reg_final:.4f}, R^2: {r2_reg_final:.4f}")
print("Final Classification Metrics:")
print(f"Accuracy: {accuracy_class_final:.4f}")

Final Regression Metrics:
MSE: 210.6077, R^2: 0.9834
Final Classification Metrics:
Accuracy: 0.9500


###4i. Сравнить результаты моделей с результатами из пункта 3

###4d. Сравнить результаты имплементированных моделей с пунктом 2

| **Задача**       | **Метрика** | **Базовый бейзлайн** | **Custom KNN** | **Изменение** |
|------------------|------------|---------------------|---------------|-------------|
| **Классификация** | Accuracy   | 0.95               | 0.95          | ➖          |
| **Регрессия**    | MSE        | 201.739248               | 210.6077          | 📉 Ухудшение  |
| **Регрессия**    | R²         | 0.984106               | 0.9841          | ➖  |

###4j. Итоговые выводы

- Метрика Accuracy осталась на уровне 0.95, как в базовом бейзлайне, так и в кастомной реализации.Классификационная модель показывает стабильные результаты. Отсутствие изменений указывает на качественную начальную предобработку данных и хороший выбор параметров.
- Метрика MSE ухудшилась с 201.739248 до 210.6077, что указывает на увеличение ошибки.
- Метрика R² осталась практически неизменной: 0.984106 в базовом бейзлайне против 0.9841 в кастомной реализации.
- Кастомная реализация регрессионной модели не смогла улучшить качество предсказаний. Возможное ухудшение связано с недостаточной гибкостью используемого метода или переобучением.
- Классификация сохранила высокое качество.
- Регрессия показала небольшое ухудшение MSE при сохранении стабильного R².
- Итоги показывают, что кастомная реализация подтверждает корректность моделей, но не превосходит начальный бейзлайн по всем метрикам.